In [1]:
import sympy as sp

In [2]:
import matplotlib.pyplot as plt

In [3]:
import sympy.physics.units.quantities as sq

In [4]:
from sympy.physics.quantum.constants import hbar

In [5]:
PSI_FUNCTION = sp.Function( "psi" )
POTENTIAL_FUNCTION = sp.Function( "V" )
TOTAL_ENERGY_SYMBOL = sq.Symbol( 'E', nonzero = True, positive = True )
MASS_SYMBOL = sq.Quantity( 'm', positive = True, nonzero = True )
#REDUCED_PLANCK_CONSTANT_SYMBOL = sq.Symbol( "hbar" )
POSITION_SYMBOL = sp.Symbol( 'x', positive = True )

def time_independent_schroedinger_equation( 
            psi = PSI_FUNCTION, 
            potential = POTENTIAL_FUNCTION, 
            total_energy = TOTAL_ENERGY_SYMBOL, 
            mass = MASS_SYMBOL, 
            reduced_planck_constant = hbar, #REDUCED_PLANCK_CONSTANT_SYMBOL, 
            position = POSITION_SYMBOL 
        ): 
    return sp.Eq( 
            ( ( -( reduced_planck_constant ** 2 ) / ( 2 * mass ) ) \
                    * sp.Derivative( psi( position ), ( position, 2 ) ) )
                    + ( potential( position ) * psi( position ) ), 
            total_energy * psi( position ) 
        )


In [6]:
set_equal = lambda to_set, value : sp.Eq( to_set, value )
both_sides = lambda equation, opreation : sp.Eq( opreation( equation.lhs ), opreation( equation.rhs ) )
equation_to_dict = lambda equation : { equation.lhs : equation.rhs }

In [7]:
hbar_eq = sp.Eq( hbar, sp.physics.units.planck / ( 2 * sp.pi ) )

In [8]:
hbar_eq

Eq(hbar, planck/(2*pi))

In [9]:
psi = PSI_FUNCTION
potential = POTENTIAL_FUNCTION
total_energy = TOTAL_ENERGY_SYMBOL
mass = MASS_SYMBOL
#reduced_planck_constant = REDUCED_PLANCK_CONSTANT_SYMBOL
x = POSITION_SYMBOL

In [10]:
k = sp.Symbol( 'k' )

In [11]:
k_squared = sp.Eq( k ** 2, ( ( 2 * total_energy * mass ) / ( hbar ** 2 ) ) )

In [12]:
k_squared

Eq(k**2, 2*m*E/hbar**2)

In [13]:
class ZeroPotential( sp.Function ): 
    @classmethod
    def eval( cls, position ):
        return 0

In [14]:
#https://docs.sympy.org/latest/modules/assumptions/index.html#querying
#https://docs.sympy.org/latest/modules/codegen.html#module-sympy.codegen.cxxnodes

class Potential( sp.Function ): 
    DEFAULT_POTENTIAL = sp.Symbol( "V_0" )
    @classmethod
    def eval( cls, position, potential = DEFAULT_POTENTIAL ): 
        return potential

In [15]:
class TunnelPotential( sp.Function ): 
    DEFAULT_WELL_LENGTH = sp.Symbol( 'L' )
    DEFAULT_POTENTIAL = Potential.DEFAULT_POTENTIAL
    DEFAULT_START = 0
    @classmethod
    def eval( cls, position, length = DEFAULT_WELL_LENGTH, 
             start = DEFAULT_START, potential = DEFAULT_POTENTIAL ): 
        if position < start or position > sp.simplify( length + start ): 
            return ZeroPotential.eval( position )
        return Potential.eval( position, potential )

In [16]:
class Stepper: 

    LEFT = "LEFT", 
    RIGHT = "RIGHT"
    
    def __init__( self, first_step, new_steps = None ): 
        self.steps = new_steps if new_steps else []
        self.steps.append( first_step )
        self.chaining = False
    
    def step_number( self, step = None ): 
        return ( len( self.steps ) - 1 ) if not step else step
    
    def last_step( self, step = None ):
        return self.steps[ self.step_number( step ) ]
    
    def _return_chain( self, step, chain ): 
        return self if ( chain or self.chaining ) else step        
    
    def add_step( self, new_step, chain = False ):
        self.steps.append( new_step )
        return self._return_chain( self.steps[ -1 ], chain )

    def operate( self, operation, step = None, chain = False ):
        self.steps.append( operation( self.last_step( step ) ) )
        return self._return_chain( self.steps[ -1 ], chain )

    def manipulate( self, operation, chain = False ): 
        return self._return_chain( self.add_step( 
                both_sides( self.last_step(), operation ) ), 
                chain 
            )
    
    def delete_step( self, step ): 
        old_step = self.steps[ step ]
        del self.steps[ step ]
        return old_step
    
    def undo( self, step = None, chain = False ): 
        return self._return_chain( 
                self.delete_step( self.step_number( step ) ), 
                chain 
            )
    
    def clone( self, from_step = None ):
        return self.branch( lambda blank : blank, from_step )
    
    def branch( self, operation = None, from_step = None ): 
        from_step = self.step_number( from_step )
        return Stepper( 
                both_sides( self.last_step( from_step ), operation ), 
                self.steps[ : self.step_number( from_step ) : ], 
            )
    
    def substitute_constant( self, constant, chain = False ):
            return self.operate( lambda step : step.subs( { constant.rhs : constant.lhs } ) )
    
    def negate_add( self, side_to_negate_add, chain = False ): 
        last_step = self.last_step()
        return self.manipulate( 
                lambda side : side + -( 
                    last_step.lhs if side_to_negate_add == Stepper.LEFT 
                            else last_step.rhs 
                        ), 
                chain 
            )
    
    def rename( self, old_symbol, new_symbol_name, chain = False ): 
        last_step = self.last_step()
        assert old_symbol in last_step.atoms() or old_symbol in last_step.atoms( sp.Function )
        return self.add_step( last_step.subs( 
            old_symbol if type( old_symbol ) == dict \
                    else { old_symbol : new_symbol_name } ), chain 
                )
    
    def chain( self, operation ):
        self.chaining = True
        operation( self )
        self.chaining = False
        return self
    
    def begin_chain( self ): 
        self.chaining = True
        return self
    
    def end_chain( self ): 
        self.chaining = False
        return self


In [17]:
k_squared

Eq(k**2, 2*m*E/hbar**2)

In [18]:
psi_region = [
        Stepper( time_independent_schroedinger_equation( potential = ZeroPotential ) ), 
        Stepper( time_independent_schroedinger_equation( potential = Potential ) ), 
        Stepper( time_independent_schroedinger_equation( potential = ZeroPotential ) )
    ]

In [19]:
psi_region[ 0 ].last_step()

Eq(-hbar**2*Derivative(psi(x), (x, 2))/(2*m), E*psi(x))

In [20]:
psi_region[ 0 ].manipulate( lambda side : ( side - psi_region[ 0 ].last_step().rhs ).simplify() )

Eq(-E*psi(x) - hbar**2*Derivative(psi(x), (x, 2))/(2*m), 0)

In [21]:
psi_region[ 0 ].manipulate( lambda side : -( side / total_energy ).simplify() )

Eq(psi(x) + hbar**2*Derivative(psi(x), (x, 2))/(2*m*E), 0)

In [22]:
psi_region[ 0 ].manipulate( lambda side : side - psi( x ) )

Eq(hbar**2*Derivative(psi(x), (x, 2))/(2*m*E), -psi(x))

In [23]:
psi_region[ 0 ].manipulate( lambda side : side * k_squared.rhs )

Eq(Derivative(psi(x), (x, 2)), -2*m*E*psi(x)/hbar**2)

In [24]:
psi_region[ 0 ].manipulate( lambda side : side / psi( x ) )

Eq(Derivative(psi(x), (x, 2))/psi(x), -2*m*E/hbar**2)

In [25]:
k_squared

Eq(k**2, 2*m*E/hbar**2)

In [26]:
psi_region[ 0 ].substitute_constant( k_squared )

Eq(Derivative(psi(x), (x, 2))/psi(x), -k**2)

In [27]:
psi_region[ 0 ].manipulate( lambda side : side * psi( x ) )

Eq(Derivative(psi(x), (x, 2)), -k**2*psi(x))

In [28]:
psi_region[ 0 ].negate_add( Stepper.RIGHT )

Eq(k**2*psi(x) + Derivative(psi(x), (x, 2)), 0)

In [29]:
# For later

psi_region[ 2 ] = psi_region[ 0 ].clone()

In [30]:
psi_region[ 2 ].last_step()

Eq(k**2*psi(x) + Derivative(psi(x), (x, 2)), 0)

In [31]:
k_0 = sp.Symbol( "k_0" )

In [32]:
psi_region[ 0 ].rename( k, k_0 )

Eq(k_0**2*psi(x) + Derivative(psi(x), (x, 2)), 0)

In [33]:
diff_sol = sp.solvers.ode.dsolve( psi_region[ 0 ].last_step().lhs, 0, ivar = x )
#ics = { psi( 0 ): 0, psi( well_length ): 0 }

In [34]:
diff_sol

Eq(psi(x), C1*exp(-I*k_0*x) + C2*exp(I*k_0*x))

In [35]:
psi_region[ 0 ].add_step( diff_sol )

Eq(psi(x), C1*exp(-I*k_0*x) + C2*exp(I*k_0*x))

In [36]:
C1 = sp.Symbol( "C1" )
C2 = sp.Symbol( "C2" )
constants = [ [ sp.Symbol( 'A' ), sp.Symbol( 'B' ) ] ]

psi_region[ 0 ].rename( C1, constants[ 0 ][ 0 ] )

Eq(psi(x), A*exp(-I*k_0*x) + C2*exp(I*k_0*x))

In [37]:
psi_region[ 0 ].rename( C2, constants[ 0 ][ 1 ] )

Eq(psi(x), A*exp(-I*k_0*x) + B*exp(I*k_0*x))

In [38]:
psi_region[ 1 ].last_step()

Eq(V_0*psi(x) - hbar**2*Derivative(psi(x), (x, 2))/(2*m), E*psi(x))

In [39]:
psi_region[ 0 ].last_step()

Eq(psi(x), A*exp(-I*k_0*x) + B*exp(I*k_0*x))

In [40]:
psi_region[ 1 ].negate_add( Stepper.RIGHT )

Eq(-E*psi(x) + V_0*psi(x) - hbar**2*Derivative(psi(x), (x, 2))/(2*m), 0)

In [41]:
psi_region[ 1 ].manipulate( lambda side : side.collect( psi(x) ) )

Eq((-E + V_0)*psi(x) - hbar**2*Derivative(psi(x), (x, 2))/(2*m), 0)

In [42]:
psi_region[ 1 ].last_step().lhs.as_two_terms()[ 1 ]

-hbar**2*Derivative(psi(x), (x, 2))/(2*m)

In [43]:
region_1_clone = psi_region[ 1 ].clone()

region_1_clone.manipulate( lambda side : side 
                - region_1_clone.last_step()
                        .lhs.as_two_terms()[ 1 ]
        )

Eq((-E + V_0)*psi(x), hbar**2*Derivative(psi(x), (x, 2))/(2*m))

In [44]:
region_1_clone.manipulate( lambda side : side / region_1_clone.last_step().lhs.as_two_terms()[ 0 ] )

Eq(psi(x), hbar**2*Derivative(psi(x), (x, 2))/(2*m*(-E + V_0)))

In [45]:
region_1_clone.manipulate( lambda side : side / region_1_clone.last_step().rhs )

Eq(2*m*(-E + V_0)*psi(x)/(hbar**2*Derivative(psi(x), (x, 2))), 1)

In [46]:
region_1_clone.manipulate( lambda side : side * sp.Derivative( psi( x ), ( x, 2 ) ) )

Eq(2*m*(-E + V_0)*psi(x)/hbar**2, Derivative(psi(x), (x, 2)))

In [47]:
region_1_clone.manipulate( lambda side : side / psi( x ) )

Eq(2*m*(-E + V_0)/hbar**2, Derivative(psi(x), (x, 2))/psi(x))

In [48]:
k_1 = sp.Symbol( "k_1" )
k_1_squared = sp.Eq( k_1 ** 2, region_1_clone.last_step().lhs )

In [49]:
k_1_squared

Eq(k_1**2, 2*m*(-E + V_0)/hbar**2)

In [50]:
region_1_clone.manipulate( lambda side : side.subs( { k_1_squared.rhs : k_1_squared.lhs } ) )

Eq(k_1**2, Derivative(psi(x), (x, 2))/psi(x))

In [51]:
region_1_clone.manipulate( lambda side : side * psi( x ) )

Eq(k_1**2*psi(x), Derivative(psi(x), (x, 2)))

In [52]:
region_1_clone.negate_add( Stepper.RIGHT )

Eq(k_1**2*psi(x) - Derivative(psi(x), (x, 2)), 0)

In [53]:
psi_region[ 1 ] = region_1_clone

In [54]:
psi_region[ 1 ].last_step()

Eq(k_1**2*psi(x) - Derivative(psi(x), (x, 2)), 0)

In [55]:
psi_1 = sp.Function( "psi_1" )

In [56]:
psi_2 = sp.Function( "psi_2" )

In [57]:
psi_region[ 1 ].rename( psi( x ), psi_1( x ) )

Eq(k_1**2*psi_1(x) - Derivative(psi_1(x), (x, 2)), 0)

In [58]:
#JUST NOTICED THAT THE SOLUTION DOES NOT CONTAINER IMAGINARY NUMBERS? WHY?

psi_region[ 1 ].add_step( sp.solvers.ode.dsolve( psi_region[ 1 ].last_step() ) )

Eq(psi_1(x), C1*exp(-k_1*x) + C2*exp(k_1*x))

In [59]:
constants.append( [ sp.Symbol( 'C' ), sp.Symbol( 'D' ) ] )

In [60]:
psi_region[ 1 ].rename( C1, constants[ 1 ][ 0 ] )

Eq(psi_1(x), C*exp(-k_1*x) + C2*exp(k_1*x))

In [61]:
psi_region[ 1 ].rename( C2, constants[ 1 ][ 1 ] )

Eq(psi_1(x), C*exp(-k_1*x) + D*exp(k_1*x))

In [62]:
k_2 = sp.Symbol( "k_2" )
psi_2_symbol = sp.Function( "psi_2" )

psi_region[ 2 ].rename( psi( x ), psi_2_symbol( x ) )

Eq(k**2*psi_2(x) + Derivative(psi_2(x), (x, 2)), 0)

In [63]:
psi_region[ 2 ].rename( k, k_2 )

Eq(k_2**2*psi_2(x) + Derivative(psi_2(x), (x, 2)), 0)

In [64]:
psi_region[ 2 ].add_step( sp.solvers.ode.dsolve( psi_region[ 2 ].last_step() ) )

Eq(psi_2(x), C1*exp(-I*k_2*x) + C2*exp(I*k_2*x))

In [65]:
constants.append( [ sp.Symbol( 'G' ) ] ) #Skipping a letter in the alphabet so there is no confusion with energy

psi_region[ 2 ].rename( C2, constants[ 2 ][ 0 ] )

Eq(psi_2(x), C1*exp(-I*k_2*x) + G*exp(I*k_2*x))

In [66]:
psi_region[ 2 ].rename( C1, 0 )

Eq(psi_2(x), G*exp(I*k_2*x))

In [67]:
#diff_sol = sp.solvers.ode.dsolve( psi_region[ 1 ].last_step().lhs, 0, ivar = x, 
#        ics = { 
#                psi_1( 0 ) : psi( 0 ), 
#                psi_1( barrier_length ) : psi_2( barrier_length ), 
#                psi_1( x ).diff( x, 1 ).subs( x, 0 ) : psi( x ).diff( x, 1 ).subs( x, 0 ), 
#                psi_1( x ).diff( x, 1 ).subs( x, barrier_length ) : psi_2( x ).diff( x, 1 ).subs( x, barrier_length )
#        }
#    )

In [68]:
flat_constants = []
for constant_set in constants: 
    flat_constants += constant_set

normalize = lambda expression, from_, to : sp.Eq( sp.integrate( expression * sp.conjugate( expression ), ( x, from_, to ) ), 1, evaluate = False )

constants_helpers = sp.solve( 
        [ equation.last_step() for equation in psi_region ], 
        flat_constants
    )

In [585]:
sp.latex( sp.Eq( sp.Integral( G * sp.conjugate( G ), ( x, 0, TunnelPotential.DEFAULT_WELL_LENGTH ) ), 1 ) )

'\\int\\limits_{0}^{L} G \\overline{G}\\, dx = 1'

In [69]:
region = []
region.append( Stepper( sp.Eq( psi_region[ 2 ].last_step().rhs, psi_region[ 1 ].last_step().rhs ) ) )

In [70]:
region[ 0 ].rename( x, TunnelPotential.DEFAULT_WELL_LENGTH )

Eq(G*exp(I*L*k_2), C*exp(-L*k_1) + D*exp(L*k_1))

In [71]:
region.append( Stepper( sp.Eq( psi_region[ 0 ].last_step().rhs, psi_region[ 1 ].last_step().rhs ) ) )

In [72]:
region[ 1 ].rename( x, 0 )

Eq(A + B, C + D)

In [73]:
region.append( Stepper( sp.Eq( sp.diff( psi_region[ 0 ].last_step().rhs, x ), sp.diff( psi_region[ 1 ].last_step().rhs, x ) ) ) )

In [74]:
region[ 2 ].rename( x, 0 )

Eq(-I*A*k_0 + I*B*k_0, -C*k_1 + D*k_1)

In [75]:
region.append( Stepper( sp.Eq( sp.diff( psi_region[ 2 ].last_step().rhs, x ), sp.diff( psi_region[ 1 ].last_step().rhs, x ) ) ) )

In [76]:
region[ 3 ].rename( x, TunnelPotential.DEFAULT_WELL_LENGTH )

Eq(I*G*k_2*exp(I*L*k_2), -C*k_1*exp(-L*k_1) + D*k_1*exp(L*k_1))

In [77]:
constants_system = [ equation.last_step() for equation in region ]

In [78]:
constants_system

[Eq(G*exp(I*L*k_2), C*exp(-L*k_1) + D*exp(L*k_1)),
 Eq(A + B, C + D),
 Eq(-I*A*k_0 + I*B*k_0, -C*k_1 + D*k_1),
 Eq(I*G*k_2*exp(I*L*k_2), -C*k_1*exp(-L*k_1) + D*k_1*exp(L*k_1))]

In [79]:
constants_solutions = sp.solve( constants_system )

In [80]:
constants_solutions = constants_solutions[ 0 ]
constants_solutions

{A: G*(k_0*(k_1 + I*k_2 + (k_1 - I*k_2)*exp(2*L*k_1)) + I*k_1*(k_1 + I*k_2 + (-k_1 + I*k_2)*exp(2*L*k_1)))*exp(-L*(k_1 - I*k_2))/(4*k_0*k_1),
 B: G*(k_0*(k_1 + I*k_2 + (k_1 - I*k_2)*exp(2*L*k_1)) + I*k_1*(-k_1 - I*k_2 + (k_1 - I*k_2)*exp(2*L*k_1)))*exp(-L*(k_1 - I*k_2))/(4*k_0*k_1),
 C: G*(k_1 - I*k_2)*exp(L*(k_1 + I*k_2))/(2*k_1),
 D: G*(k_1 + I*k_2)*exp(-L*(k_1 - I*k_2))/(2*k_1)}

In [575]:
sp.latex( constants_solutions[ D ] )

G*(k_1 + I*k_2)*exp(-L*(k_1 - I*k_2))/(2*k_1)

In [376]:
constants_solutions[ A ]

G*(k_0*(k_1 + I*k_2 + (k_1 - I*k_2)*exp(2*L*k_1)) + I*k_1*(k_1 + I*k_2 + (-k_1 + I*k_2)*exp(2*L*k_1)))*exp(-L*(k_1 - I*k_2))/(4*k_0*k_1)

In [81]:
#solve_g = region[ 3 ].clone()
#solve_g.rename( constants[ 1 ][ 0 ], constants_solutions[ constants[ 1 ][ 0 ] ] )
#solve_g.rename( constants[ 1 ][ 1 ], constants_solutions[ constants[ 1 ][ 1 ] ] )
#solve_g.manipulate( lambda side : side * 2 )
#solve_g.manipulate( lambda side : side / sp.exp( sp.I * TunnelPotential.DEFAULT_WELL_LENGTH * k_2 ) )
#solve_g.add_step( sp.Eq( solve_g.last_step().lhs, solve_g.last_step().rhs.collect( sp.Symbol( 'G' ) ) ) )
#solve_g.negate_add( Stepper.RIGHT )
#solve_g.add_step( sp.Eq( solve_g.last_step().lhs.collect( sp.Symbol( 'G' ) ), solve_g.last_step().rhs ) )

In [82]:
G = sp.Symbol( 'G', real = True )

In [576]:
normalize( G, 0, TunnelPotential.DEFAULT_WELL_LENGTH )

Eq(G*L*conjugate(G), 1)

In [ ]:
g_normalized = Stepper( normalize( psi_region[ 2 ].last_step().rhs, 0, TunnelPotential.DEFAULT_WELL_LENGTH ) )

In [ ]:
g_normalized.last_step()

Eq(I*sqrt(-I*(-k_2 + conjugate(k_2))*exp(I*L*conjugate(k_2))/(exp(I*L*k_2) - exp(I*L*conjugate(k_2))))*exp(I*L*k_2)*conjugate(sqrt(-I*(-k_2 + conjugate(k_2))*exp(I*L*conjugate(k_2))/(exp(I*L*k_2) - exp(I*L*conjugate(k_2)))))/(-k_2*exp(I*L*conjugate(k_2)) + exp(I*L*conjugate(k_2))*conjugate(k_2)) - I*sqrt(-I*(-k_2 + conjugate(k_2))*exp(I*L*conjugate(k_2))/(exp(I*L*k_2) - exp(I*L*conjugate(k_2))))*conjugate(sqrt(-I*(-k_2 + conjugate(k_2))*exp(I*L*conjugate(k_2))/(exp(I*L*k_2) - exp(I*L*conjugate(k_2)))))/(-k_2 + conjugate(k_2)), 1)

In [ ]:
g_normalized.add_step( sp.Eq( g_normalized.last_step().lhs.together(), 1 ) )

Eq(I*sqrt(-I*(-k_2 + conjugate(k_2))*exp(I*L*conjugate(k_2))/(exp(I*L*k_2) - exp(I*L*conjugate(k_2))))*(exp(I*L*k_2) - exp(I*L*conjugate(k_2)))*exp(-I*L*conjugate(k_2))*conjugate(sqrt(-I*(-k_2 + conjugate(k_2))*exp(I*L*conjugate(k_2))/(exp(I*L*k_2) - exp(I*L*conjugate(k_2)))))/(-k_2 + conjugate(k_2)), 1)

In [ ]:
numerator, denomonator = g_normalized.last_step().lhs.as_numer_denom()

In [ ]:
numerator

I*sqrt(-I*(-k_2 + conjugate(k_2))*exp(I*L*conjugate(k_2))/(exp(I*L*k_2) - exp(I*L*conjugate(k_2))))*(exp(I*L*k_2) - exp(I*L*conjugate(k_2)))*conjugate(sqrt(-I*(-k_2 + conjugate(k_2))*exp(I*L*conjugate(k_2))/(exp(I*L*k_2) - exp(I*L*conjugate(k_2)))))

In [ ]:
denomonator

(-k_2 + conjugate(k_2))*exp(I*L*conjugate(k_2))

In [ ]:
g_normalized.manipulate( lambda side : side * denomonator )

Eq(I*sqrt(-I*(-k_2 + conjugate(k_2))*exp(I*L*conjugate(k_2))/(exp(I*L*k_2) - exp(I*L*conjugate(k_2))))*(exp(I*L*k_2) - exp(I*L*conjugate(k_2)))*conjugate(sqrt(-I*(-k_2 + conjugate(k_2))*exp(I*L*conjugate(k_2))/(exp(I*L*k_2) - exp(I*L*conjugate(k_2))))), (-k_2 + conjugate(k_2))*exp(I*L*conjugate(k_2)))

In [ ]:
div_out = numerator.as_two_terms()[ 1 ].as_two_terms()[ 1 ].as_two_terms()[ 0 ]

In [ ]:
div_out

exp(I*L*k_2) - exp(I*L*conjugate(k_2))

In [ ]:
g_normalized.manipulate( lambda side : side / div_out )

Eq(I*sqrt(-I*(-k_2 + conjugate(k_2))*exp(I*L*conjugate(k_2))/(exp(I*L*k_2) - exp(I*L*conjugate(k_2))))*conjugate(sqrt(-I*(-k_2 + conjugate(k_2))*exp(I*L*conjugate(k_2))/(exp(I*L*k_2) - exp(I*L*conjugate(k_2))))), (-k_2 + conjugate(k_2))*exp(I*L*conjugate(k_2))/(exp(I*L*k_2) - exp(I*L*conjugate(k_2))))

In [93]:
real_imag = g_normalized.last_step().lhs.as_real_imag()

In [94]:
real_imag

(0, Abs(G)**2)

In [95]:
to_replace = g_normalized.last_step().lhs / sp.I

In [96]:
to_replace

G*conjugate(G)

In [97]:
g_normalized.add_step( g_normalized.last_step().subs( { to_replace : real_imag[ 1 ] } ) )

Eq(I*Abs(G)**2, (-k_2 + conjugate(k_2))*exp(I*L*conjugate(k_2))/(exp(I*L*k_2) - exp(I*L*conjugate(k_2))))

In [98]:
g_normalized.manipulate( lambda side : side / sp.I )

Eq(Abs(G)**2, -I*(-k_2 + conjugate(k_2))*exp(I*L*conjugate(k_2))/(exp(I*L*k_2) - exp(I*L*conjugate(k_2))))

In [99]:
g_normalized.manipulate( lambda side : sp.sqrt( side ) )

Eq(Abs(G), sqrt(-I*(-k_2 + conjugate(k_2))*exp(I*L*conjugate(k_2))/(exp(I*L*k_2) - exp(I*L*conjugate(k_2)))))

In [100]:
import numpy as np

In [101]:
constant_subs = { constant_ : constants_solutions[ constant_ ].subs( { sp.Symbol( 'G' ) : g_normalized.last_step().rhs } ) for constant_ in constants_solutions }

In [102]:
constant_subs = { constant_ : constant_subs[ constant_ ].simplify().refine( sp.Q.real( k_2 ) ).refine( sp.Q.positive( k_2 ) ) for constant_ in constant_subs }

In [103]:
constant_subs

{A: sqrt(I*(k_2 - conjugate(k_2))*exp(I*L*conjugate(k_2))/(exp(I*L*k_2) - exp(I*L*conjugate(k_2))))*(k_0*(k_1 + I*k_2 + (k_1 - I*k_2)*exp(2*L*k_1)) + I*k_1*(k_1 + I*k_2 + (-k_1 + I*k_2)*exp(2*L*k_1)))*exp(-L*(k_1 - I*k_2))/(4*k_0*k_1),
 B: sqrt(I*(k_2 - conjugate(k_2))*exp(I*L*conjugate(k_2))/(exp(I*L*k_2) - exp(I*L*conjugate(k_2))))*(k_0*(k_1 + I*k_2 + (k_1 - I*k_2)*exp(2*L*k_1)) - I*k_1*(k_1 + I*k_2 + (-k_1 + I*k_2)*exp(2*L*k_1)))*exp(-L*(k_1 - I*k_2))/(4*k_0*k_1),
 C: sqrt(I*(k_2 - conjugate(k_2))*exp(I*L*conjugate(k_2))/(exp(I*L*k_2) - exp(I*L*conjugate(k_2))))*(k_1 - I*k_2)*exp(L*(k_1 + I*k_2))/(2*k_1),
 D: sqrt(I*(k_2 - conjugate(k_2))*exp(I*L*conjugate(k_2))/(exp(I*L*k_2) - exp(I*L*conjugate(k_2))))*(k_1 + I*k_2)*exp(-L*(k_1 - I*k_2))/(2*k_1)}

In [104]:
psi_region[ 0 ].last_step()

Eq(psi(x), A*exp(-I*k_0*x) + B*exp(I*k_0*x))

In [105]:
psi_region[ 1 ].last_step()

Eq(psi_1(x), C*exp(-k_1*x) + D*exp(k_1*x))

In [106]:
psi_region[ 2 ].last_step()

Eq(psi_2(x), G*exp(I*k_2*x))

In [107]:
A = sp.Symbol( 'A' )
B = sp.Symbol( 'B' )
C = sp.Symbol( 'C' )
D = sp.Symbol( 'D' )
G = sp.Symbol( 'G' )


In [108]:
psi_region[ 0 ].rename( A, constant_subs[ A ], chain=True ).rename( B, constant_subs[ B ] )

Eq(psi(x), sqrt(I*(k_2 - conjugate(k_2))*exp(I*L*conjugate(k_2))/(exp(I*L*k_2) - exp(I*L*conjugate(k_2))))*(k_0*(k_1 + I*k_2 + (k_1 - I*k_2)*exp(2*L*k_1)) - I*k_1*(k_1 + I*k_2 + (-k_1 + I*k_2)*exp(2*L*k_1)))*exp(-L*(k_1 - I*k_2))*exp(I*k_0*x)/(4*k_0*k_1) + sqrt(I*(k_2 - conjugate(k_2))*exp(I*L*conjugate(k_2))/(exp(I*L*k_2) - exp(I*L*conjugate(k_2))))*(k_0*(k_1 + I*k_2 + (k_1 - I*k_2)*exp(2*L*k_1)) + I*k_1*(k_1 + I*k_2 + (-k_1 + I*k_2)*exp(2*L*k_1)))*exp(-L*(k_1 - I*k_2))*exp(-I*k_0*x)/(4*k_0*k_1))

In [109]:
psi_region[ 0 ].last_step().simplify().refine()#.refine( sp.Q.positive( k_2 ) )

Eq(psi(x), sqrt(I*(k_2 - conjugate(k_2))*exp(I*L*conjugate(k_2))/(exp(I*L*k_2) - exp(I*L*conjugate(k_2))))*((k_0*(k_1 + I*k_2 + (k_1 - I*k_2)*exp(2*L*k_1)) - I*k_1*(k_1 + I*k_2 + (-k_1 + I*k_2)*exp(2*L*k_1)))*exp(L*(k_1 - I*k_2) + 2*I*k_0*x) + (k_0*(k_1 + I*k_2 + (k_1 - I*k_2)*exp(2*L*k_1)) + I*k_1*(k_1 + I*k_2 + (-k_1 + I*k_2)*exp(2*L*k_1)))*exp(L*(k_1 - I*k_2)))*exp(-2*L*(k_1 - I*k_2) - I*k_0*x)/(4*k_0*k_1))

In [110]:
psi_region[ 1 ].rename( C, constant_subs[ C ], chain=True ).rename( D, constant_subs[ D ] )

Eq(psi_1(x), sqrt(I*(k_2 - conjugate(k_2))*exp(I*L*conjugate(k_2))/(exp(I*L*k_2) - exp(I*L*conjugate(k_2))))*(k_1 - I*k_2)*exp(L*(k_1 + I*k_2))*exp(-k_1*x)/(2*k_1) + sqrt(I*(k_2 - conjugate(k_2))*exp(I*L*conjugate(k_2))/(exp(I*L*k_2) - exp(I*L*conjugate(k_2))))*(k_1 + I*k_2)*exp(-L*(k_1 - I*k_2))*exp(k_1*x)/(2*k_1))

In [111]:
psi_region[ 1 ].last_step().simplify()

Eq(psi_1(x), sqrt(I*(k_2 - conjugate(k_2))*exp(I*L*conjugate(k_2))/(exp(I*L*k_2) - exp(I*L*conjugate(k_2))))*((k_1 - I*k_2)*exp(2*L*k_1) + (k_1 + I*k_2)*exp(2*k_1*x))*exp(-L*(k_1 - I*k_2) - k_1*x)/(2*k_1))

In [112]:
psi_region[ 2 ].rename( G, g_normalized.last_step().rhs )

Eq(psi_2(x), sqrt(-I*(-k_2 + conjugate(k_2))*exp(I*L*conjugate(k_2))/(exp(I*L*k_2) - exp(I*L*conjugate(k_2))))*exp(I*k_2*x))

In [113]:
k_squared

Eq(k**2, 2*m*E/hbar**2)

In [114]:
psi_numerical = [ current_psi_region.clone() for current_psi_region in psi_region ]

In [115]:
psi_numerical[ 0 ].rename( k_2, k_0 )

Eq(psi(x), sqrt(I*(k_0 - conjugate(k_0))*exp(I*L*conjugate(k_0))/(exp(I*L*k_0) - exp(I*L*conjugate(k_0))))*(k_0*(I*k_0 + k_1 + (-I*k_0 + k_1)*exp(2*L*k_1)) - I*k_1*(I*k_0 + k_1 + (I*k_0 - k_1)*exp(2*L*k_1)))*exp(-L*(-I*k_0 + k_1))*exp(I*k_0*x)/(4*k_0*k_1) + sqrt(I*(k_0 - conjugate(k_0))*exp(I*L*conjugate(k_0))/(exp(I*L*k_0) - exp(I*L*conjugate(k_0))))*(k_0*(I*k_0 + k_1 + (-I*k_0 + k_1)*exp(2*L*k_1)) + I*k_1*(I*k_0 + k_1 + (I*k_0 - k_1)*exp(2*L*k_1)))*exp(-L*(-I*k_0 + k_1))*exp(-I*k_0*x)/(4*k_0*k_1))

In [116]:
#psi_numerical[ 0 ].last_step().simplify().refine( sp.Q.real( k_0 ) )

In [125]:
psi_numerical[ 0 ].add_step( sp.Eq( psi_numerical[ 0 ].last_step().lhs, 
                            psi_numerical[ 0 ].last_step().rhs.subs( { k_1 : sp.sqrt( k_1_squared.rhs ) } ).simplify() ) )

Eq(psi(x), sqrt(2)*sqrt(I*(k_0 - conjugate(k_0))*exp(I*L*conjugate(k_0))/(exp(I*L*k_0) - exp(I*L*conjugate(k_0))))*(-(sqrt(2)*I*sqrt(m)*sqrt(-E + V_0)*(sqrt(2)*sqrt(m)*sqrt(-E + V_0) + hbar*I*k_0 + (-sqrt(2)*sqrt(m)*sqrt(-E + V_0) + hbar*I*k_0)*exp(2*sqrt(2)*sqrt(m)*L*sqrt(-E + V_0)/hbar)) - hbar*k_0*(sqrt(2)*sqrt(m)*sqrt(-E + V_0) + hbar*I*k_0 + (sqrt(2)*sqrt(m)*sqrt(-E + V_0) - hbar*I*k_0)*exp(2*sqrt(2)*sqrt(m)*L*sqrt(-E + V_0)/hbar)))*exp((L*(sqrt(2)*sqrt(m)*sqrt(-E + V_0) - hbar*I*k_0) + 2*hbar*I*k_0*x)/hbar) + (sqrt(2)*I*sqrt(m)*sqrt(-E + V_0)*(sqrt(2)*sqrt(m)*sqrt(-E + V_0) + hbar*I*k_0 + (-sqrt(2)*sqrt(m)*sqrt(-E + V_0) + hbar*I*k_0)*exp(2*sqrt(2)*sqrt(m)*L*sqrt(-E + V_0)/hbar)) + hbar*k_0*(sqrt(2)*sqrt(m)*sqrt(-E + V_0) + hbar*I*k_0 + (sqrt(2)*sqrt(m)*sqrt(-E + V_0) - hbar*I*k_0)*exp(2*sqrt(2)*sqrt(m)*L*sqrt(-E + V_0)/hbar)))*exp(L*(sqrt(2)*sqrt(m)*sqrt(-E + V_0) - hbar*I*k_0)/hbar))*exp(-(2*L*(sqrt(2)*sqrt(m)*sqrt(-E + V_0) - hbar*I*k_0) + hbar*I*k_0*x)/hbar)/(8*hbar*sqrt(m)*k

In [126]:
#psi_numerical[ 0 ].add_step( sp.Eq( psi_numerical[ 0 ].last_step().lhs, 
#psi_numerical[ 0 ].last_step().rhs.subs( { k_0 : sp.sqrt( k_squared.rhs ) } )

nan

In [131]:
psi_numerical[ 0 ].add_step( psi_numerical[ 0 ].last_step().refine() )

Eq(psi(x), sqrt(I*(k_0 - conjugate(k_0))*exp(I*L*conjugate(k_0))/(exp(I*L*k_0) - exp(I*L*conjugate(k_0))))*(k_0*(I*k_0 + k_1 + (-I*k_0 + k_1)*exp(2*L*k_1)) - I*k_1*(I*k_0 + k_1 + (I*k_0 - k_1)*exp(2*L*k_1)))*exp(-L*(-I*k_0 + k_1))*exp(I*k_0*x)/(4*k_0*k_1) + sqrt(I*(k_0 - conjugate(k_0))*exp(I*L*conjugate(k_0))/(exp(I*L*k_0) - exp(I*L*conjugate(k_0))))*(k_0*(I*k_0 + k_1 + (-I*k_0 + k_1)*exp(2*L*k_1)) + I*k_1*(I*k_0 + k_1 + (I*k_0 - k_1)*exp(2*L*k_1)))*exp(-L*(-I*k_0 + k_1))*exp(-I*k_0*x)/(4*k_0*k_1))

In [149]:
a = 0

In [151]:
k_0_squared = k_squared.subs( { k : k_0 } )

In [156]:
k_0_squared

Eq(k_0**2, 2*m*E/hbar**2)

In [189]:
psi_numerical[ 0 ].add_step( psi_numerical[ 0 ].last_step().simplify() )

Eq(psi(x), sqrt(I*(k_0 - conjugate(k_0))*exp(I*L*conjugate(k_0))/(exp(I*L*k_0) - exp(I*L*conjugate(k_0))))*((k_0*(I*k_0 + k_1 + (-I*k_0 + k_1)*exp(2*L*k_1)) - I*k_1*(I*k_0 + k_1 + (I*k_0 - k_1)*exp(2*L*k_1)))*exp(L*(I*k_0 - k_1) + 2*I*k_0*x) + (k_0*(I*k_0 + k_1 + (-I*k_0 + k_1)*exp(2*L*k_1)) + I*k_1*(I*k_0 + k_1 + (I*k_0 - k_1)*exp(2*L*k_1)))*exp(L*(I*k_0 - k_1)))*exp(-I*k_0*x)/(4*k_0*k_1))

In [374]:
psi_numerical[ 0 ].last_step().rhs.subs( { k_0 : sp.sqrt( k_0_squared.rhs ) } )

nan

In [407]:
epon = lambda k_ : sp.exp( sp.I*TunnelPotential.DEFAULT_WELL_LENGTH*k_ )

In [410]:
undefined_term = ( psi_numerical[ 0 ].last_step().rhs.separate() * 4*k_0*k_1 ).as_two_terms()[ 0 ].args[ 0 ]

In [411]:
undefined_term

I*(k_0 - conjugate(k_0))*exp(I*L*conjugate(k_0))/(exp(I*L*k_0) - exp(I*L*conjugate(k_0)))

In [415]:
epon_ck_0_subs = epon( sp.conjugate( k_0 ) )

In [417]:
epon_ck_0_subs

exp(I*L*conjugate(k_0))

In [416]:
epon_k_0_subs = epon( k_0 )

In [427]:
sp.I*epon_k_0_subs

I*exp(I*L*k_0)

In [482]:
undefined_term.args_cnc()

[[I,
  1/(exp(I*L*k_0) - exp(I*L*conjugate(k_0))),
  k_0 - conjugate(k_0),
  exp(I*L*conjugate(k_0))],
 []]

In [544]:
#dir( 
numer_denom = undefined_term.args_cnc()[ 0 ]#[ 1 ].as_numer_denom()[ 1 ]
q = numer_denom[ 1 ].as_numer_denom()[ 1 ].together()#.collect( epon_k_0_subs )
# )#.subs( { epon_ck_0_subs : epon_k_0_subs } )

In [545]:
q

exp(I*L*k_0) - exp(I*L*conjugate(k_0))

In [546]:
q = sp.Mul( sp.Add( 1, -1, evaluate = False ), q.as_two_terms()[ 1 ], evaluate = False )

In [547]:
q

(-1 + 1)*exp(I*L*k_0)

In [548]:
term_ = numer_denom[ 2 ].as_two_terms()[ 0 ] 

qq = sp.Add( term_, -term_, evaluate = False )

In [549]:
qq

-k_0 + k_0

In [550]:
qq = sp.Mul( qq, numer_denom[ 3 ].subs( { sp.conjugate( k_0 ) : k_0 } ), evaluate = False )

In [551]:
qq

(-k_0 + k_0)*exp(I*L*k_0)

In [553]:
qqq = q * k_0 

In [554]:
qqq

k_0*(-1 + 1)*exp(I*L*k_0)

In [564]:
sp.Mul( qqq, q ** -1, evaluate=False )

(exp(-I*L*k_0)/(-1 + 1))*(k_0*(-1 + 1)*exp(I*L*k_0))

In [567]:
sp.sqrt( sp.I * qqq/q )

sqrt(I*k_0)

In [480]:
with sp.assuming( k_0_squared, sp.Q.real( total_energy ), sp.Q.positive( total_energy ), 
              sp.Q.real( mass ), sp.Q.positive( mass ), sp.Q.nonzero( mass ), 
              sp.Q.real( total_energy ), sp.Q.positive( total_energy ), sp.Q.nonzero( total_energy ), 
              total_energy > Potential.DEFAULT_POTENTIAL, 
              sp.Q.real( Potential.DEFAULT_POTENTIAL ), sp.Q.positive( Potential.DEFAULT_POTENTIAL ), 
              sp.Q.nonzero( Potential.DEFAULT_POTENTIAL ), 
              sp.Q.positive( hbar ), sp.Q.real( hbar ), sp.Q.nonzero( hbar ) ): 
    print( sp.ask( sp.Q.positive( k_0 ) ) )

None


In [424]:
a

zoo*(k_0 - conjugate(k_0))*exp(I*L*k_0)

In [475]:
psi_numerical[ 0 ].last_step().rhs.extract_branch_factor()#.subs( { undefined_term : 0 } )

(sqrt(I*(k_0 - conjugate(k_0))*exp(I*L*conjugate(k_0))/(exp(I*L*k_0) - exp(I*L*conjugate(k_0))))*((k_0*(I*k_0 + k_1 + (-I*k_0 + k_1)*exp(2*L*k_1)) - I*k_1*(I*k_0 + k_1 + (I*k_0 - k_1)*exp(2*L*k_1)))*exp(L*(I*k_0 - k_1) + 2*I*k_0*x) + (k_0*(I*k_0 + k_1 + (-I*k_0 + k_1)*exp(2*L*k_1)) + I*k_1*(I*k_0 + k_1 + (I*k_0 - k_1)*exp(2*L*k_1)))*exp(L*(I*k_0 - k_1)))*exp(-I*k_0*x)/(4*k_0*k_1),
 0)

In [443]:
#dir( 
psi_numerical[ 0 ].last_step().rhs.cancel().refine()
# )

(-I*k_0**2*sqrt(I*k_0*exp(I*L*conjugate(k_0))/(exp(I*L*k_0) - exp(I*L*conjugate(k_0))) - I*exp(I*L*conjugate(k_0))*conjugate(k_0)/(exp(I*L*k_0) - exp(I*L*conjugate(k_0))))*exp(L*k_1)*exp(I*L*k_0)*exp(2*I*k_0*x) - I*k_0**2*sqrt(I*k_0*exp(I*L*conjugate(k_0))/(exp(I*L*k_0) - exp(I*L*conjugate(k_0))) - I*exp(I*L*conjugate(k_0))*conjugate(k_0)/(exp(I*L*k_0) - exp(I*L*conjugate(k_0))))*exp(L*k_1)*exp(I*L*k_0) + I*k_0**2*sqrt(I*k_0*exp(I*L*conjugate(k_0))/(exp(I*L*k_0) - exp(I*L*conjugate(k_0))) - I*exp(I*L*conjugate(k_0))*conjugate(k_0)/(exp(I*L*k_0) - exp(I*L*conjugate(k_0))))*exp(-L*k_1)*exp(I*L*k_0)*exp(2*I*k_0*x) + I*k_0**2*sqrt(I*k_0*exp(I*L*conjugate(k_0))/(exp(I*L*k_0) - exp(I*L*conjugate(k_0))) - I*exp(I*L*conjugate(k_0))*conjugate(k_0)/(exp(I*L*k_0) - exp(I*L*conjugate(k_0))))*exp(-L*k_1)*exp(I*L*k_0) + 2*k_0*k_1*sqrt(I*k_0*exp(I*L*conjugate(k_0))/(exp(I*L*k_0) - exp(I*L*conjugate(k_0))) - I*exp(I*L*conjugate(k_0))*conjugate(k_0)/(exp(I*L*k_0) - exp(I*L*conjugate(k_0))))*exp(L*k_1)*

In [351]:
str( psi_numerical[ 0 ].last_step().rhs )

'Eq(psi(x), sqrt(I*(k_0 - conjugate(k_0))*exp(I*L*conjugate(k_0))/(exp(I*L*k_0) - exp(I*L*conjugate(k_0))))*((k_0*(I*k_0 + k_1 + (-I*k_0 + k_1)*exp(2*L*k_1)) - I*k_1*(I*k_0 + k_1 + (I*k_0 - k_1)*exp(2*L*k_1)))*exp(L*(I*k_0 - k_1) + 2*I*k_0*x) + (k_0*(I*k_0 + k_1 + (-I*k_0 + k_1)*exp(2*L*k_1)) + I*k_1*(I*k_0 + k_1 + (I*k_0 - k_1)*exp(2*L*k_1)))*exp(L*(I*k_0 - k_1)))*exp(-I*k_0*x)/(4*k_0*k_1))'

In [120]:
k_squared.rhs

2*m*E/hbar**2

In [122]:
position_axis = np.linspace( 0, 1 )